<a href="https://colab.research.google.com/github/GHunor/CMO_NN/blob/main/CMO_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
from zipfile import ZipFile
import os

from pathlib import Path

In [ ]:
labels_path = "/content/dataset_all.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./DS_o")

labels_path = "/content/DS_o/dataset_labels.zip"
#zip_path = keras.utils.get_file(origin=uri, fname="data_labels0.zip")
zip_file = ZipFile(labels_path)
zip_file.extractall("./labels")

labels_path = "/content/DS_o/dataset_myo.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./myo")

labels_path = "/content/DS_o/dataset_opt.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./op")

In [ ]:
myodim = 8
labdim = 15
optdim = 9

In [ ]:
sizeL = [];
sizeM = [];
sizeO = [];

In [ ]:
L_ADRS = [];
hp = './labels'
p = '/'
for child in sorted(Path(hp).iterdir()):
    if child.is_file():
        L_ADRS.append(child.name)
labels = [];
for adrs in L_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#df = pd.read_csv(hp + p + adrs, header=None)
    labels.append(df)
    sizeL.append(df.shape)

In [ ]:
M_ADRS = [];
hp = './myo'
p = '/'
for child in sorted(Path(hp).iterdir()):
    if child.is_file():
        M_ADRS.append(child.name)
myo = [];
for adrs in M_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#= pd.read_csv(hp + p + adrs, header=None)
    myo.append(df)
    sizeM.append(df.shape)

In [ ]:
O_ADRS = [];
hp = './op'
p = '/'
for child in sorted(Path(hp).iterdir()):
    if child.is_file():
        O_ADRS.append(child.name)
op = [];
for adrs in O_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#df = pd.read_csv(hp + p + adrs, header=None)
    op.append(df)
    sizeO.append(df.shape)

In [ ]:
"""
  df = pd.read_csv('./labels/' + ADRS[0])
  labels = [];
  labels.append(df)
  df = pd.read_csv('./labels/' + ADRS[1])
  labels.append(df)
  print(labels)
  print(labels[1])
  print(df)
"""

"\n  df = pd.read_csv('./labels/' + ADRS[0])\n  labels = [];\n  labels.append(df)\n  df = pd.read_csv('./labels/' + ADRS[1])\n  labels.append(df)\n  print(labels)\n  print(labels[1])\n  print(df)\n"

In [ ]:
#ADRS = pd.read_csv(csv_path)
#df = pd.read_csv(ADRS(i))


In [ ]:
# create dataset and labels

#model.reset_states() before feeding the new dataset and calling model.fit()
# only myo 15 moves

# opto too 15 moves

# Have a CNN run thorugh 10 datappoint (one data point represents 10 ns)
# 1. only CNN (17->18-20)
# 2. only LSTM
# 3. Both

In [ ]:
print(np.size(myo))

65


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [ ]:
#opt lstm
x_val = op[0]
y_val = labels[0]

sequence_length = 20
step = 1
batch_size = 64
learning_rate = 0.001
epochs = 5

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch


x_test = op[1]
y_test = labels[1]

DS_T = [];
for i, M in enumerate(op[2:]):
  if np.size(M, 0)<sequence_length:
    continue
  i = i+2
  print(i)
  x_train = M
  ishape = []
  ishape.append(1)
  ishape.append(np.size(M, 1))
  y_train = labels[i]
  oshape = []
  oshape.append(1)
  oshape.append(np.size(labels[i], 1))
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      x_train,
      y_train,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )
  DS_T.append(dataset_train)

path_checkpoint5 =  "model5_checkpoint.h5"

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)



inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
LSoutputs = keras.layers.Dense(oshape[1])(lstm_out)

model5 = keras.Model(inputs=inputs, outputs=LSoutputs)
model5.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model5.summary()

modelckpt_callback5 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint5,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

# will be repeated
for dataset_train in DS_T:
  print(dataset_train)
  history = model5.fit(
      dataset_train,
      epochs=epochs,
      validation_data=dataset_val,
      callbacks=[es_callback, modelckpt_callback5],
      verbose = 0
  )
  model5.reset_states() 


dataset_test = keras.preprocessing.timeseries_dataset_from_array(
      x_test,
      y_test,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )

[score1, score2] = model2.evaluate(dataset_test, verbose = 1) 

print('Test loss:', score1) 
print('Test error:', score2)

#OPT LSTM

In [ ]:
x_val = myo[0]
y_val = labels[0]

sequence_length = 20
step = 1
batch_size = 64
learning_rate = 0.001
epochs = 5

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch

In [ ]:
"""
i = 1;
len = np.size(labels[i], 0)
print(len)
print(labels[0])
"""

'\ni = 1;\nlen = np.size(labels[i], 0)\nprint(len)\nprint(labels[0])\n'

In [ ]:
x_test = myo[1]
y_test = labels[1]

In [ ]:
# will be repeated
DS_T = [];
for i, M in enumerate(myo[2:]):
  if np.size(M, 0)<sequence_length:
    continue
  i = i+2
  print(i)
  x_train = M
  ishape = []
  ishape.append(1)
  ishape.append(np.size(M, 1))
  y_train = labels[i]
  oshape = []
  oshape.append(1)
  oshape.append(np.size(labels[i], 1))
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      x_train,
      y_train,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )
  DS_T.append(dataset_train)



2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [ ]:
# will be repeated



In [ ]:

path_checkpoint1 =  "model1_checkpoint.h5"
path_checkpoint2 =  "model2_checkpoint.h5"
path_checkpoint3 =  "model3_checkpoint.h5"

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback1 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint1,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback2 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint2,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback3 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint3,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

In [ ]:
#models
#input_shape = (batch_size, sequence_length, ishape[1])
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
#inputs = keras.layers.Input(shape=(ishape[0], ishape[1]))
#inputs = keras.layers.Input(shape=(1,17))


lstm_out = keras.layers.LSTM(32)(inputs)
LSoutputs = keras.layers.Dense(oshape[1])(lstm_out)

"""
cnn_out1 = keras.layers.Conv1D(32, 1, activation='relu',input_shape=(inputs.shape[1], inputs.shape[2]))(inputs)
CNNoutputs = keras.layers.Dense(oshape[1])(cnn_out1)
model1 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model1.summary()
"""

#x = tf.ones((batch_size, inputs.shape[1], inputs.shape[1]))
#y = model1(x)
#model1.summary()

model2 = keras.Model(inputs=inputs, outputs=LSoutputs)
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model2.summary()

"""
lstm_out2 = keras.layers.LSTM(32)(cnn_out1)
COMBoutputs = keras.layers.Dense(oshape[1])(lstm_out2)
model3 = keras.Model(inputs=inputs, outputs=COMBoutputs)
model3.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model3.summary()
"""

# model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
# model.summary()


modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint1,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

# will be repeated
for dataset_train in DS_T:
  print(dataset_train)
  history2 = model2.fit(
      dataset_train,
      epochs=epochs,
      validation_data=dataset_val,
      callbacks=[es_callback, modelckpt_callback2],
      verbose = 0
  )
  model2.reset_states() 

  #x_test = x_val,
#y_test = y_val,

dataset_test = keras.preprocessing.timeseries_dataset_from_array(
      x_test,
      y_test,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )

[score1, score2] = model2.evaluate(dataset_test, verbose = 1) 

#print(score)
#print(np.size(score))
print('Test loss:', score1) 
print('Test error:', score2)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 8)]           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                5248      
                                                                 
 dense_1 (Dense)             (None, 15)                495       
                                                                 
Total params: 5,743
Trainable params: 5,743
Non-trainable params: 0
_________________________________________________________________
<BatchDataset element_spec=(TensorSpec(shape=(None, None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None, 15), dtype=tf.float64, name=None))>

Epoch 1: val_loss did not improve from 0.04120

Epoch 2: val_loss did not improve from 0.04120

Epoch 3: val_loss did not improve from 0.04120

Epoch 4: val_loss did no

1/1 [==============================] - 0s 44ms/step - loss: 0.0636 - root_mean_squared_error: 0.2522
Test loss: 0.06361017376184464
Test error: 0.2522105574607849


In [ ]:
print(sizeL)

[(118, 15), (39, 15), (39, 15), (38, 15), (38, 15), (39, 15), (39, 15), (38, 15), (38, 15), (38, 15), (38, 15), (38, 15), (39, 15), (38, 15), (38, 15), (38, 15), (38, 15), (38, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (38, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (38, 15), (39, 15), (39, 15), (38, 15), (38, 15), (38, 15), (37, 15), (37, 15), (38, 15), (38, 15), (12, 15), (38, 15), (39, 15), (39, 15), (38, 15), (39, 15), (38, 15), (39, 15), (39, 15), (39, 15), (40, 15), (39, 15), (40, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (39, 15), (37, 15), (39, 15), (39, 15), (39, 15)]


In [ ]:
print(sizeM)

[(118, 8), (39, 8), (39, 8), (38, 8), (38, 8), (39, 8), (39, 8), (38, 8), (38, 8), (38, 8), (38, 8), (38, 8), (39, 8), (38, 8), (38, 8), (38, 8), (38, 8), (38, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (38, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (38, 8), (39, 8), (39, 8), (38, 8), (38, 8), (38, 8), (37, 8), (37, 8), (38, 8), (38, 8), (12, 8), (38, 8), (39, 8), (39, 8), (38, 8), (39, 8), (38, 8), (39, 8), (39, 8), (39, 8), (40, 8), (39, 8), (40, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (39, 8), (37, 8), (39, 8), (39, 8), (39, 8)]


In [ ]:
print(sizeO)

[(118, 9), (39, 9), (39, 9), (38, 9), (38, 9), (39, 9), (39, 9), (38, 9), (38, 9), (38, 9), (38, 9), (38, 9), (39, 9), (38, 9), (38, 9), (38, 9), (38, 9), (38, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (38, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (38, 9), (39, 9), (39, 9), (38, 9), (38, 9), (38, 9), (37, 9), (37, 9), (38, 9), (38, 9), (12, 9), (38, 9), (39, 9), (39, 9), (38, 9), (39, 9), (38, 9), (39, 9), (39, 9), (39, 9), (40, 9), (39, 9), (40, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (39, 9), (37, 9), (39, 9), (39, 9), (39, 9)]


In [ ]:
#preprocess time series into "pics"
len = np.size(myo[0], 0)
h = 8
TrainDx = np.empty([0, h, myodim, 1]).astype('float32')
TrainDy = np.empty([0, labdim, 1]).astype('float32')
TestDx = np.empty([0, h, myodim, 1]).astype('float32')
TestDy = np.empty([0, labdim, 1]).astype('float32')
for j in range(len-h):
  #TD.append(myo[i:(h+1), 1:3])
  arr =  myo[0][j:j+h]
  arr = np.reshape(arr, [1,  h, myodim, 1])
  TestDx = np.append(TestDx, arr, axis=0)

  arr =  labels[0][j+h-1:j+h]
  arr = np.reshape(arr, [1,  labdim, 1])
  TestDy = np.append(TestDy, arr, axis=0)

for k, L in  enumerate(myo[1:]):
  if np.size(L, 0)<h:
    continue
  len2 = np.size(L, 0)
  for j in range(len2-h):
    #TD.append(myo[i:(h+1), 1:3])
    arr =  np.asarray(L[j:j+h])
    arr = np.reshape(arr, [1,  h, myodim, 1])

    TrainDx = np.append(TrainDx, arr, axis=0)

    arr =  labels[k+1][j+h-1:j+h]

    arr = np.reshape(arr, [1,  labdim, 1])
    TrainDy = np.append(TrainDy, arr, axis=0)

#print(TrainDx[0])
print("TrainDx size:")
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
print(np.size(TrainDx[0], 0))
#print(np.size(TrainDx, 2))
#print(TrainDy[0])
print(np.size(TrainDy, 0))
#print(np.size(TrainDy, 1))
#print(labels[0])


TrainDx size:
123584
1931
8
1931


In [ ]:
"""
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
TrainDx = np.asarray(TrainDx)
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
arr = np.array(TrainDx)
print(np.size(arr))
print(np.size(arr, 0))
TrainDx = TrainDx.reshape(np.size(TrainDx, 0), h, 8, 1)
"""

'\nprint(np.size(TrainDx))\nprint(np.size(TrainDx, 0))\nTrainDx = np.asarray(TrainDx)\nprint(np.size(TrainDx))\nprint(np.size(TrainDx, 0))\narr = np.array(TrainDx)\nprint(np.size(arr))\nprint(np.size(arr, 0))\nTrainDx = TrainDx.reshape(np.size(TrainDx, 0), h, 8, 1)\n'

In [ ]:
#print(TrainDx.dtypes)
#print(TrainDy.dtypes)

In [ ]:

TrainDy = np.asarray(TrainDy).astype('float32')

In [ ]:
TestDx = np.asarray(TestDx).astype('float32')
TestDx = TestDx.reshape(np.size(TestDx, 0), np.size(TestDx, 1), np.size(TestDx, 2), 1)

TestDy = np.asarray(TestDy).astype('float32')

In [ ]:
input_shape = TrainDx.shape
print(input_shape)
Kern1 = [2,1,1]
Kern2 = [3,3,2]
Kern = Kern2

(1931, 8, 8, 1)


In [ ]:

print(input_shape)
inputs = keras.layers.Input(shape=input_shape[1:])
cnn_out1 = keras.layers.Conv2D(16, kernel_size = (3, Kern[0]), activation = 'relu', input_shape = input_shape[1:])(inputs)
cnn_out2 = keras.layers.Conv2D(32, kernel_size = (3, Kern[1]), activation = 'relu')(cnn_out1)
cnn_out3 = keras.layers.MaxPooling2D(pool_size = (2, Kern[2]))(cnn_out2)
cnn_out4 = keras.layers.Flatten()(cnn_out3)
cnn_out5 = keras.layers.Dense(64, activation = 'relu')(cnn_out4)
CNNoutputs = keras.layers.Dense(oshape[1], activation = 'softmax')(cnn_out5)
model1 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model1.summary()



(1931, 8, 8, 1)
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 8, 8, 1)]         0         
                                                                 
 conv2d (Conv2D)             (None, 6, 6, 16)          160       
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                           

In [ ]:
for i in range(1, 11):
  history1 = model1.fit(
  TrainDx, TrainDy,
  batch_size = batch_size,
  epochs = 20,
  verbose = 1,
  validation_data = (TestDx, TestDy),
  callbacks=[es_callback, modelckpt_callback1]
)

Epoch 1/20
31/31 [==============================] - ETA: 0s - loss: 0.4233 - root_mean_squared_error: 0.6506
Epoch 1: val_loss improved from inf to 0.46542, saving model to model1_checkpoint.h5
31/31 [==============================] - 5s 12ms/step - loss: 0.4233 - root_mean_squared_error: 0.6506 - val_loss: 0.4654 - val_root_mean_squared_error: 0.6822
Epoch 2/20
30/31 [============================>.] - ETA: 0s - loss: 0.4136 - root_mean_squared_error: 0.6431
Epoch 2: val_loss improved from 0.46542 to 0.46260, saving model to model1_checkpoint.h5
31/31 [==============================] - 0s 5ms/step - loss: 0.4137 - root_mean_squared_error: 0.6432 - val_loss: 0.4626 - val_root_mean_squared_error: 0.6801
Epoch 3/20
17/31 [===============>..............] - ETA: 0s - loss: 0.4107 - root_mean_squared_error: 0.6409
Epoch 3: val_loss improved from 0.46260 to 0.46213, saving model to model1_checkpoint.h5
31/31 [==============================] - 0s 5ms/step - loss: 0.4127 - root_mean_squared_err

In [ ]:
[score1, score2] = model1.evaluate(TestDx, TestDy, verbose = 1) 

print('Test loss:', score1) 
print('Test error:', score2)

4/4 [==============================] - 0s 17ms/step - loss: 0.4610 - root_mean_squared_error: 0.6790
Test loss: 0.4610118269920349
Test error: 0.6789785027503967


In [ ]:
MO = []

for i, L in enumerate(myo):
  T = np.concatenate((myo[i], op[i]), axis=1,  dtype='float32', casting="same_kind")
  print(T.shape)
  MO.append(T)


(118, 17)
(39, 17)
(39, 17)
(38, 17)
(38, 17)
(39, 17)
(39, 17)
(38, 17)
(38, 17)
(38, 17)
(38, 17)
(38, 17)
(39, 17)
(38, 17)
(38, 17)
(38, 17)
(38, 17)
(38, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(38, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(38, 17)
(39, 17)
(39, 17)
(38, 17)
(38, 17)
(38, 17)
(37, 17)
(37, 17)
(38, 17)
(38, 17)
(12, 17)
(38, 17)
(39, 17)
(39, 17)
(38, 17)
(39, 17)
(38, 17)
(39, 17)
(39, 17)
(39, 17)
(40, 17)
(39, 17)
(40, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(39, 17)
(37, 17)
(39, 17)
(39, 17)
(39, 17)


In [ ]:
# concat myo with opt
x_val = MO[0]
x_val = np.asarray(x_val)
#print(np.size(x_val, 0))
#print(np.size(x_val, 1))
y_val = labels[0]
y_val = np.asarray(y_val)
#print(np.size(y_val, 0))
#print(np.size(y_val, 1))

sequence_length = 20
step = 1
batch_size = 64
learning_rate = 0.001
epochs = 5

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch
#print(inputs.shape[0])
#print(inputs.shape[1])

x_test = MO[1]
y_test = labels[1]


DS_T = [];
for i, M in enumerate(MO[2:]):
  if np.size(M, 0)<sequence_length:
    continue
  i = i+2
  #print(i)
  x_train = M
  ishape = []
  ishape.append(1)
  ishape.append(np.size(M, 1))
  y_train = labels[i]
  oshape = []
  oshape.append(1)
  oshape.append(np.size(labels[i], 1))
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      x_train,
      y_train,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )
  DS_T.append(dataset_train)

path_checkpoint3 =  "model3_checkpoint.h5"
path_checkpoint4 =  "model4_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback3 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint3,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback4 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint4,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)


In [ ]:

inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
LSoutputs = keras.layers.Dense(oshape[1])(lstm_out)


model4 = keras.Model(inputs=inputs, outputs=LSoutputs)
model4.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model4.summary()

for dataset_train in DS_T:
  print(dataset_train)
  history4 = model4.fit(
      dataset_train,
      epochs=epochs,
      validation_data=dataset_val,
      callbacks=[es_callback, modelckpt_callback4],
      verbose=0
  )
  model4.reset_states() 


dataset_test = keras.preprocessing.timeseries_dataset_from_array(
      x_test,
      y_test,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )

[score1, score2] = model4.evaluate(dataset_test, verbose = 1) 

print('Test loss:', score1) 
print('Test error:', score2)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 20, 17)]          0         
                                                                 
 lstm_17 (LSTM)              (None, 32)                6400      
                                                                 
 dense_19 (Dense)            (None, 15)                495       
                                                                 
Total params: 6,895
Trainable params: 6,895
Non-trainable params: 0
_________________________________________________________________
<BatchDataset element_spec=(TensorSpec(shape=(None, None, 17), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float64, name=None))>

Epoch 1: val_loss did not improve from 0.04006

Epoch 2: val_loss did not improve from 0.04006

Epoch 3: val_loss did not improve from 0.04006

Epoch 4: val_loss did 

1/1 [==============================] - 0s 61ms/step - loss: 0.0634 - root_mean_squared_error: 0.2519
Test loss: 0.06343284994363785
Test error: 0.25185877084732056


In [ ]:

len = np.size(myo[0], 0)
h = 8
TrainDx = np.empty([0, h, myodim, 1]).astype('float32')
TrainDy = np.empty([0, labdim, 1]).astype('float32')
TestDx = np.empty([0, h, myodim, 1]).astype('float32')
TestDy = np.empty([0, labdim, 1]).astype('float32')
for j in range(len-h):
  #TD.append(myo[i:(h+1), 1:3])
  arr =  myo[0][j:j+h]
  arr = np.reshape(arr, [1,  h, myodim, 1])
  TestDx = np.append(TestDx, arr, axis=0)

  arr =  labels[0][j+h-1:j+h]
  arr = np.reshape(arr, [1,  labdim, 1])
  TestDy = np.append(TestDy, arr, axis=0)

for k, L in  enumerate(myo[1:]):
  if np.size(L, 0)<h:
    continue
  len2 = np.size(L, 0)
  for j in range(len2-h):
    arr =  np.asarray(L[j:j+h])
    arr = np.reshape(arr, [1,  h, myodim, 1])
    TrainDx = np.append(TrainDx, arr, axis=0)
    arr =  labels[k+1][j+h-1:j+h]
    arr = np.reshape(arr, [1,  labdim, 1])
    TrainDy = np.append(TrainDy, arr, axis=0)


TrainDx = np.asarray(TrainDx)
TrainDx = TrainDx.reshape(np.size(TrainDx, 0), np.size(TrainDx, 1), np.size(TrainDx, 2), 1)
print("TrainDx.shape")
print(TrainDx.shape)

TrainDy = np.asarray(TrainDy)

TestDx = np.asarray(TestDx)
TestDx = TestDx.reshape(np.size(TestDx, 0), np.size(TestDx, 1), np.size(TestDx, 2), 1)

TestDy = np.asarray(TestDy)

input_shape = (np.size(TrainDx, 0), np.size(TrainDx, 1), np.size(TrainDx, 2), 1)
Kern1 = [2,1,1]
Kern2 = [3,3,2]
Kern = Kern1

inputs = keras.layers.Input(shape=input_shape[1:])
cnn_out1 = keras.layers.Conv2D(16, kernel_size = (3, Kern[0]), activation = 'relu', input_shape = input_shape[1:])(inputs)
cnn_out2 = keras.layers.Conv2D(32, kernel_size = (3, Kern[1]), activation = 'relu')(cnn_out1)
cnn_out3 = keras.layers.MaxPooling2D(pool_size = (2, Kern[2]))(cnn_out2)
cnn_out4 = keras.layers.Flatten()(cnn_out3)
cnn_out5 = keras.layers.Dense(64, activation = 'relu')(cnn_out4)
CNNoutputs = keras.layers.Dense(oshape[1], activation = 'softmax')(cnn_out5)
model3 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model3.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model3.summary()

history3 = model3.fit(
  TrainDx, TrainDy,
  batch_size = batch_size,
  epochs = epochs,
  verbose = 1,
  validation_data = (TestDx, TestDy),
  callbacks=[es_callback, modelckpt_callback3]
)

[score1, score2] = model3.evaluate(TestDx, TestDy, verbose = 1) 

print('Test loss:', score1) 
print('Test error:', score2)

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 8, 8, 1)]         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 6, 7, 16)          112       
                                                                 
 conv2d_7 (Conv2D)           (None, 4, 7, 32)          1568      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 448)               0         
                                                                 
 dense_15 (Dense)            (None, 64)                28736     
                                                          

In [ ]:
"""
import keras
from keras.datasets import mnist
from keras.sampleEducbaModels import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as sampleEducba
import numpy as np
#Loading the mnist dataset for the sample purpose
(TrainV1, TrainV2), (TestV1, TestV2) = mnist.load_data()
#modify the existing data set for our requirement of the sampleEducbaModel.
rImg, columnsOfImage = 28, 28
if sampleEducba.image_data_format() == 'channels_first':
  TrainV1 = TrainV1.reshape(TrainV1.shape[0], 1, rImg, columnsOfImage)
  TestV1 = TestV1.reshape(TestV1.shape[0], 1, rImg, columnsOfImage)
  input_shape = (1, rImg, columnsOfImage)
else:
  TrainV1 = TrainV1.reshape(TrainV1.shape[0], rImg, columnsOfImage, 1)
  TestV1 = TestV1.reshape(TestV1.shape[0], rImg, columnsOfImage, 1)
  input_shape = (rImg, columnsOfImage, 1)
TrainV1 = TrainV1.astype('float32')
TestV1 = TestV1.astype('float32')
TrainV1 /= 255
TestV1 /= 255
TrainV2 = keras.utils.to_categorical(TrainV2, 10)
TestV2 = keras.utils.to_categorical(TestV2, 10)
#Model creation
Model = Sequential()
Model.add(Conv2D(32, kernel_size = (3, 3),
activation = 'relu', input_shape = input_shape))
Model.add(Conv2D(64, (3, 3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size = (2, 2)))
Model.add(Dropout(0.25)) 
Model.add(Flatten())
Model.add(Dense(128, activation = 'relu'))
Model.add(Dropout(0.5))
Model.add(Dense(10, activation = 'softmax'))
# Model compilation
Model.compile(loss = keras.losses.categorical_crossentropy,
optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])
# training the sampleEducbaModel by using fit function
Model.fit(
  TrainV1, TrainV2,
  batch_size = 128,
  epochs = 12,
  verbose = 1,
  validation_data = (TestV1, TestV2)
)